In [ ]:
# 0 - legitimate
# 1 - phishing
# 2 - suspicious

In [84]:
# packages
import pandas as pd
from urllib.parse import urlparse,urlencode
import re
from bs4 import BeautifulSoup
import requests
import whois
import urllib.request
from datetime import datetime
import time
import socket
from urllib.error import HTTPError
from cython.parallel import prange

In [67]:
from datetime import datetime
itime = datetime.now()
for i in prange(0,10000):
    pass
ftime = datetime.now()
print(ftime-itime)

0:00:00.004000


In [2]:
#computer_raw_data=pd.read_csv("H:/CDAC_PROJECT/PhishingWebsiteDetection/legitimate_urls.txt",header=None,names=["urls"])
#computer_raw_data.head(10)


In [36]:
computer_raw_data=pd.read_csv("1000-phishing.txt",header=None,names=['urls'])
#computer_raw_data

In [3]:
#rows = len(computer_raw_data)

In [4]:
#computer_raw_data["urls"][0]

In [85]:
cd = None
class FeatureExtraction:
    def __init__(self):
        pass
    
    def getProtocol(self,url):
        return urlparse(url).scheme
    
    def getDomain(self,url):
        return urlparse(url).netloc
    
    def getPath(self,url):
        return urlparse(url).path
    
    def havingIP(self,url):
        """If the domain part has IP then it is phishing otherwise legitimate"""
        match=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  #IPv4
                    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  #IPv4 in hexadecimal
                    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)     #Ipv6
        if match:
            #print match.group()
            return 1            # phishing
        else:
            #print 'No matching pattern found'
            return 0            # legitimate
    
    def long_url(self,url):
        """This function is defined in order to differntiate website based on the length of the URL"""
        if len(url) < 54:
            return 0            # legitimate
        elif len(url) >= 54 and len(url) <= 75:
            return 2            # suspicious
        else:
            return 1            # phishing
    
    def have_at_symbol(self,url):
        """This function is used to check whether the URL contains @ symbol or not"""
        if "@" in url:
            return 1            # phishing
        else:
            return 0            # legitimate
    
    def redirection(self,url):
        """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
        if "//" in urlparse(url).path:
            return 1            # phishing
        else:
            return 0            # legitimate
        
    def prefix_suffix_separation(self,url):
        """If the domain has '-' symbol then it is considered as phishing site"""
        if "-" in urlparse(url).netloc:
            return 1            # phishing
        else:
            return 0            # legitimate
        
    def sub_domains(self,url):
        """If the url has more than 3 dots then it is a phishing"""
        if url.count(".") < 3:
            return 0            # legitimate
        elif url.count(".") == 3:
            return 2            # suspicious
        else:
            return 1            # phishing
        
    def shortening_service(self,url):
        """Tiny URL -> phishing otherwise legitimate"""
        match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)
        if match:
            return 1               # phishing
        else:
            return 0               # legitimate
        
    """
    def google_index(self,url):
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'
        headers = { 'User-Agent' : user_agent}
        query = {'q': 'info:' + url}
        google = "https://www.google.com/search?" + urlencode(query)
        #data = requests.get(google, headers=headers,proxies=proxies)
        data = requests.get(google,headers=headers)
        data.encoding = 'ISO-8859-1'
        soup = BeautifulSoup(str(data.content), "html.parser")
        try:
            check = soup.find(id="rso").find("div").find("div").find("h3").find("a")
            if soup.find(id="rso").find("div").find("div").find("h3").find("a").find("href" != None):
                href = check['href']
                return 0 # indexed
            else:
                return 1
        except AttributeError:
            return 1 # indexed
        #print("Waiting " + str(seconds) + " seconds until checking next URL.\n")
        #time.sleep(float(seconds))
    """
    """
    def abnormal_url(self,url):
        dns = 0
        #domain_name = ""
        try:
            #domain = urlparse(url).netloc
            #print(domain)
            domain_name = whois.whois(urlparse(url).netloc)
            #print(domain_name)
        except:
            dns = 1
        
        if dns == 1:
            return 1 # phishing
        else:
            hostname=domain_name.domain_name
            #match=re.search(hostname,url)
            if hostname in url:
                return 0 # legitimate
            else:
                return 1 # phishing
    """
    
    def web_traffic(self,url):
        try:
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        except TypeError:
            return 1
        except HTTPError:
            return 2
        rank= int(rank)
        if (rank<100000):
            return 0
        else:
            return 2
        
    def domain_registration_length(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1      #phishing
        else:
            expiration_date = domain_name.expiration_date
            today = time.strftime('%Y-%m-%d')
            today = datetime.strptime(today, '%Y-%m-%d')
            if expiration_date is None:
                return 1
            elif type(expiration_date) is list or type(today) is list :
                return 2     #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
            else:
                creation_date = domain_name.creation_date
                expiration_date = domain_name.expiration_date
                if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                    try:
                        creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                        expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                    except:
                        return 2
                registration_length = abs((expiration_date - today).days)
                if registration_length / 365 <= 1:
                    return 1 #phishing
                else:
                    return 0 # legitimate
            
    def age_domain(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            creation_date = domain_name.creation_date
            expiration_date = domain_name.expiration_date
            if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                try:
                    creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                    expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                except:
                    return 2
            if ((expiration_date is None) or (creation_date is None)):
                return 1
            elif ((type(expiration_date) is list) or (type(creation_date) is list)):
                return 2
            else:
                ageofdomain = abs((expiration_date - creation_date).days)
                if ((ageofdomain/30) < 6):
                    return 1
                else:
                    return 0
     
    
    def dns_record(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
            #rint(domain_name)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            return 0
        
   
    def statistical_report(self,url):
        hostname = url
        h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
        z = int(len(h))
        if z != 0:
            y = h[0][1]
            hostname = hostname[y:]
            h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
            z = int(len(h))
            if z != 0:
                hostname = hostname[:h[0][0]]
        url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
        try:
            ip_address = socket.gethostbyname(hostname)
            ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
        except:
            return 1

        if url_match:
            return 1
        else:
            return 0
        
    def https_token(self,url):
        match=re.search('https://|http://',url)
        try:
            if match.start(0)==0 and match.start(0) is not None:
                url=url[match.end(0):]
                match=re.search('http|https',url)
                if match:
                    return 1
                else:
                    return 0
        except:
            return 1




In [86]:
#features
protocol = []
domain = []
path = []
having_ip = []
len_url = []
having_at_symbol = []
redirection_symbol = []
prefix_suffix_separation = []
sub_domains = []
tiny_url = []
abnormal_url = []
web_traffic = []
domain_registration_length = []
dns_record = []
statistical_report = []
age_domain = []
http_tokens = []
#google_index = []

### 0 - legitimate
### 1 - phishing
### 2 - suspicious

In [87]:
# object creation
fe = FeatureExtraction()
rows = len(computer_raw_data["urls"])

for i in range(0,rows):
    url=computer_raw_data["urls"][i]
    print(i ),print(url)
    protocol.append(fe.getProtocol(url))
    path.append(fe.getPath(url))
    domain.append(fe.getDomain(url))
    having_ip.append(fe.havingIP(url))
    len_url.append(fe.long_url(url))
    having_at_symbol.append(fe.have_at_symbol(url))
    redirection_symbol.append(fe.redirection(url))
    prefix_suffix_separation.append(fe.prefix_suffix_separation(url))
    sub_domains.append(fe.sub_domains(url))
    tiny_url.append(fe.shortening_service(url))
    web_traffic.append(fe.web_traffic(url))
    domain_registration_length.append(fe.domain_registration_length(url))
    dns_record.append(fe.dns_record(url))
    statistical_report.append(fe.statistical_report(url))
    age_domain.append(fe.age_domain(url))
    http_tokens.append(fe.https_token(url))
    #google_index.append(fe.google_index(url))
    #abnormal_url.append(fe.abnormal_url(url))
    
#print(domain)
#print(protocol)
#print(ip)
#print(len_url)
#print(google_index)
#print(abnormal_url)
#print(web_traffic)
#print(domain_registration_length)
#print(age_domain)
#print(dns_record)
#print(statistical_report)

0
http://asesoresvelfit.com/media/datacredito.co/
1
http://caixa.com.br.fgtsagendesaqueconta.com/consulta8523211/principal.php?seg=11
2
http://hissoulreason.com/js/homepage/home/
3
http://unauthorizd.newebpage.com/webapps/66fbf/
4
http://133.130.103.10/23/
5
http://dj00.co.vu/css/?bsoul=Qg@xIHW%//yh/en/?i=34453&amp;i=34453
6
http://133.130.103.10/21/logar/
7
http://httpssicredi.esy.es/servico/sicredi/validarclientes/mobi/index.php
8
http://gamesaty.ga/wp-content///yh/en/?i=31416&amp;i=31416
9
http://luxuryupgradepro.com/ymailNew/ymailNew/
10
http://133.130.103.10/1/
11
http://133.130.103.10/24/sicredi/psmlId/31/paneid/index.htm
12
http://smscaixaacesso.hol.es
13
http://133.130.103.10/7/SIIBC/siwinCtrl.php
14
http://tinyurl.com/kjmmw57
15
http://wrightlandscapes.org/no/T/Y1.html
16
http://mautic.eto-cms.ru/themes/goldstar/mtbonline/newmandt/
17
http://ginatringali.com//al/alibaba21012015/alibaba21012015/666/index.html
18
https://staticmail.000webhostapp.com/
19
http://umeda.com.br/bba/B

138
http://asa-direct.ru/bitrix/updates/servi/fr/doss/
139
http://pascasarjana.uniba-bpn.ac.id/administrator/components/com_joomlaupdate/redire.html
140
http://hb.redllink.com/.ars/empresas/login.htm
141
http://hb.redllink.com/.ars/empresas/login.htm?val=true
142
http://xenangdaiduong.com.vn/wp-config/Sign/
143
http://tizianabeauty.ch/wp-content/plugins/Manage/ar4d65/checking/check/Support/ID-NUMB389/myaccount/signin/
144
http://zemindar.lt/wp-content/languages/dpbx/index.php
145
http://www.crielv.com.br/images/start2.php
146
http://www.sagittariusglobal.com//portal/cache/Cielo/ambiente-seguro.php
147
http://www.sagittariusglobal.com//portal/cache/Cielo/index.html
148
https://goo.gl/MMT7sp
149
http://abuv.com.br/cliente.conectado40/atendimento_pessoa_fisica/criptografado/index1.php
150
http://voetudoazul.serveuser.com/tudoazul/cadastre.php
151
https://www.rustmaster.com.br/ml/next1.php
152
https://www.rustmaster.com.br/ml/
153
http://30horasdigital.myvnc.com/30horasdigitalmodulodesegur

268
http://sitesumo.com/outlook-app/main.html
269
http://www.r129motoring.com/SLWerks.com/wp-includes/js/tinymce/themes/modern/default/
270
http://novavidasamambaia.com.br/admin/ckeditor/plugins/link/linkedin.com/
271
http://academica.pt/U/mail.htm?_pageLabel=page_logonform&amp;cmd=LOB=RBGLogon&amp;secured_page=
272
http://academica.pt/U/mail.htm?cmd=LOB=RBGLogon&amp;_pageLabel=page_logonform&amp;secured_page
273
http://www.bestsearch.info/
274
http://innovateapp.ru/inj/1au/03.php
275
http://ovtoc.com/wp-content/themes/Astra/images/webapps/mpp/home/?cmd=_home&amp;dispatch=5885d80a13c0db1f8e&amp;ee=7a9fa49f42effcc5f28cb61b89e02e59
276
http://www.ellepimmobiliare.it/www.santandernet.com.br-br/pessoa-fisica/santander-van-gogh/aplicativos/index1.php
277
http://www.ellepimmobiliare.it/www.santandernet.com.br-br/pessoa-fisica/santander-van-gogh/aplicativos/index1.php?17
278
http://env-9019617.jelastic.regruhosting.ru/libraries/legacy/access/system.php
279
http://ambrosiabanquethall.com/mtdoc

399
http://portalfgts.com/caixa.gov.br/pages/inter/index.php
400
http://103.236.137.117/caixa.gov.br/pages/inter/
401
http://normansfinejewelry.com/ASDE/QWAUTHW/asb.co.nz/index.htm
402
http://inhomecareidaho.com/wp-includes/news/dropbaka/07b4089c45eabdd3349d4d8ba473301a/pindex.php
403
http://mainepta.org/eme/gm/excel/excel/
404
http://caixa.suportedeconsultafgtsliberado.com/consulta9104336/index.php?consulta=DLB4RNNPDLB4JO0U-DLB4JO0UJO0U-SJCEJO0U
405
http://www.saiastrology.com/sender/Leaves/filewords/index.php
406
http://notify-action-acct.esy.es/recovery-chekpoint-login.html
407
http://fangtasiaphonesex.com/Godsgrace/Dropfile/
408
http://latitudeinfra.com/admin/uploade/01myfs/dada9b67bf59f6aadf1357adbdfc409a/
409
https://boxmail-fullupgrade.com/secured/
410
http://www.michaelsmolski.com/0w1/Owa/
411
http://miprofe.cl/wp-admin/network/rdr/4/
412
http://kesenianjawa.undip.ac.id/wp-content/plugins/ADOBE_YEAR_2017/index.htm
413
https://u.nu/zfy
414
http://tinyurl.com/lhzy2me
415
http://c

512
http://mashtaresort.com/mail.htm?_pageLabel=page_logonform&amp;cmd=LOB
513
http://pixelnam.net/skonsko/BDBBB4/BDBBB/BDBBB/dpbx/index.php
514
http://www.unitedkingdomimmigrationservices.org/jj_jp/e3dc325482ee637ca0bda89a9370b486/
515
http://grannyflat-s.com.au/army/domainz/domainz/review/?email=abuse@hotmail.com
516
http://ip-160-153-224-253.ip.secureserver.net/
517
http://inativasfgtsativas.site/tnt/caixa.gov.br/pages/inter/envio1.php
518
http://piewerks.com/chinex/ogs/draft/mail.htm?_pagelabel=page_logonform&amp;cmd=lob=rbglogon&amp;secured_page=
519
http://www.facebooknotsz.gq/f4/123456.mob
520
http://www.facebooknotsz.gq/f4/
521
http://savagtolldoc.us/done/index.html
522
http://sivasosb.org.tr/webvc/index.php?option=com_content&amp;view=article&amp;id=79:premio-itau-de-financas-sustentaveis&amp;catid=8&amp;Itemid=191322
523
http://simer.com.tr/tescoonlineupdate.secures/
524
http://www.antiquehostel.ro/logs/sxxs/dda6e20944a66809a09f4c577a882cac/verification/verification.php
525
h

629
http://omahcorp.co.id/dx/012962f7ce4c0854415e6bd264535560/
630
http://i-house.se/wp-trackback/en/?83199a1c6dc81bf0be202d27a83fa6ea=&amp;_Email=&amp;cmd=_home&amp;dispatch=1f8f0b48dbdb92536e9f355fe5c05c1210088af655be910b6ee49393c9da77a2
631
http://cic.com.jo/libraries/openid/Auth/OpenID/system/login.php?id=/Secure/promocao.do?=9J8GI90IN/IJSY563/TLR1TV8VVXO0VXT4FDDPW3QGMKFSRITL5MM/IUVESKPPJKT1Y9QVBIC/4TTVDNHIA6IU2FAV21LMLGNKPFH3YV33PW/4LINXO8SQO5NQ60ESQ3EHJWLISOMJMMO07C9VL3NA8B3DMH8ELMX7JJ
632
http://ad-blockeds.com/--/www.impots.gouv.fr/file/dd14aa66803d699fee4b16424002d39d/
633
http://ad-blockeds.com/--/www.impots.gouv.fr/file/cf036cb9af9ed6833821231c37a85a51/
634
http://town-el.com/images/metalware/xkzero/docusign.com/ldcs/index.php
635
http://www.scoreyourexback.com/aol.net/www.aol.com/
636
http://nanotechnopolis.com/validdocument/f4d2dfb73a486890c95a3b59d9090b5f/
637
http://saughallmassie.org/we/seniorpeoplemeet/v3/login.html
638
http://www.paypal-resolveds.com.informations-paym

727
https://c4cqk784.caspio.com/dp.asp?AppKey=a6d75000a94cc2e429f041f0a6aa
728
http://caixaonlinefgts.hostkda.com/pages/
729
http://telegraphycoursemorse.com/cielo/
730
https://tinyurl.com/mt5hquu
731
http://contasinativascaixafgts.name/caixa.gov.br/pages/inter/index.php
732
http://signin.amazon.co.uk-prime.form-unsuscribe.id-7816.nproofing.com.au/ren/op/
733
http://primoframe.com/amber/
734
http://tinyurl.com/katfbes
735
http://consultabeneficiosmirror01.esy.es/Consulta/acesso/
736
http://tinyurl.com/n8g7wy8
737
http://adpportal.hol.es/adp.payrollchanges/
738
http://ts4.96.lt/i/
739
http://diariapromotionspecial.com/OFERTAS11910apc/produto.php?linkcompleto=smartphone-samsung-galaxy-j5-sm-j500m-ds-dourado-dual-chip-android-5-1-lollipop-4g-wi-fi-16gb/3216990/pr&amp;id=1
740
http://innamorata.com.ar/melodyfriespitchment/Docusign/index.php
741
http://promotion-smart.com/iphone5s.html
742
http://designaxis.net/secure-doc/DropNewVasion/Fresh/
743
http://ornamentmania.com/shell/gdoc-secure/g

837
http://fb-securityloginfo405txt.esy.es/recovery-chekpoint-login.html
838
http://protecfb-error-404.esy.es/recovery-chekpoint-login.html
839
http://fbscurerescekpoint23.esy.es/recovery-chekpoint-login.html
840
https://samaalreem.com/pedrodoc/index.html
841
http://alpa.co.za/modules/mod_feed/pay/index.html
842
https://greatwesterncatskills.com/z/
843
http://agenmurahgrosir.com/page/Bofa/security.php
844
http://ecutrack.com/box1/dropbx.php?docs=view&amp;readable-filetypes&amp;docpdfxls=3af55772835a104a0a81ad9948d254c03af55772835a104a0a81ad9948d254c0
845
http://primoframe.com/jimmy/
846
http://wpcgwadar.com/iyawoee/aptgd/
847
http://mobile-free.metulwx.com/mobile/impaye/e413ea8afcc080a62afbbd4a53ae798f/
848
http://www.chevysportsman.com/xxxxx/Aol/aol.htm
849
http://freestyle.d934713.u-telcom.net/Boa_Fud_2016/home/login.php?cmd=login_submit&amp;id=c175082131a50bed04f6b83823c92df5c175082131a50bed04f6b83823c92df5&amp;session=c175082131a50bed04f6b83823c92df5c175082131a50bed04f6b83823c92df5

957
http://unitedgpdev.com/fonts/paglg/personas/
958
http://asyastil.com/cats/3/1/06/87zrazsazr4a9raz/
959
http://web-appsantander.com/pessoajuridica/index.html
960
http://fb-securityloginfo406txt.esy.es/recovery-chekpoint-login.html
961
http://fb-securityloginfo407txt.esy.es/recovery-chekpoint-login.html
962
https://www.paypal-customerfeedback.com/?c88n7v5znbn297v&lng=en_US
963
http://sign-app-privacy.hol.es/logins.php
964
http://fedex.com-0289.systems/d4f9b7694dfe6fb2424e2702b25e4c8f/
965
http://fedex.com-0289.systems/c020032173ca2ec8e69b523158bb67ee/
966
http://fedex.com-0289.systems/track.php
967
http://fedex.56690.services/rd01.htm
968
http://pleasantonassembly.org/vendor_extra/pear/xs/
969
http://cover-acct1029.esy.es/index.html
970
http://suspended-fb.hol.es/logins.php
971
http://freestyle.d934713.u-telcom.net/Boa_Fud_2016/home/login.php?cmd=login_submit&amp;id=da6b43395b9eb70818cef8dc78a3602eda6b43395b9eb70818cef8dc78a3602e&amp;session=da6b43395b9eb70818cef8dc78a3602eda6b43395b

In [88]:
label = []
for i in range(0,rows):
    label.append(1)

In [89]:
d={'Protocol':pd.Series(protocol),'Domain':pd.Series(domain),'Path':pd.Series(path),'Having_IP':pd.Series(having_ip),
   'URL_Length':pd.Series(len_url),'Having_@_symbol':pd.Series(having_at_symbol),
   'Redirection_//_symbol':pd.Series(redirection_symbol),'Prefix_suffix_separation':pd.Series(prefix_suffix_separation),
   'Sub_domains':pd.Series(sub_domains),'tiny_url':pd.Series(tiny_url),'web_traffic' : pd.Series(web_traffic) ,
   'domain_registration_length':pd.Series(domain_registration_length),'dns_record':pd.Series(dns_record),
   'statistical_report':pd.Series(statistical_report),'age_domain':pd.Series(age_domain),'http_tokens':pd.Series(http_tokens),
   'label':pd.Series(label)}
data=pd.DataFrame(d)
data

Domain  Having_@_symbol  Having_IP  \
0                       asesoresvelfit.com                0          0   
1    caixa.com.br.fgtsagendesaqueconta.com                0          0   
2                        hissoulreason.com                0          0   
3                unauthorizd.newebpage.com                0          0   
4                           133.130.103.10                0          1   
5                               dj00.co.vu                1          0   
6                           133.130.103.10                0          1   
7                      httpssicredi.esy.es                0          0   
8                              gamesaty.ga                0          0   
9                     luxuryupgradepro.com                0          0   
10                          133.130.103.10                0          1   
11                          133.130.103.10                0          1   
12                   smscaixaacesso.hol.es                0          0   
13                          133.130.103.10                0          1   
14                             tinyurl.com                0          0   
15                    wrightlandscapes.org                0          0   
16                       mautic.eto-cms.ru                0          0   
17                        ginatringali.com                0          0   
18            staticmail.000webhostapp.com                0          0   
19                            umeda.com.br                0          0   
20                      krishworldwide.com                0          0   
21                             yahoo.co.in                0          0   
22                          www.avcc.ac.in                0          0   
23                         craig-hallum.ga                0          0   
24         www.verification-mobile-nab.com                0          0   
25             verification-mobile-nab.com                0          0   
26             www.justpiddlinboutique.com                0          0   
27                     vcsolutionscorp.com                0          0   
28                             almasur.com                0          0   
29                      asesoresvelfit.com                0          0   
..                                     ...              ...        ...   
968                 pleasantonassembly.org                0          0   
969                  cover-acct1029.esy.es                0          0   
970                    suspended-fb.hol.es                0          0   
971         freestyle.d934713.u-telcom.net                0          0   
972                       rooferexpert.com                1          0   
973                        demo.369usa.com                0          0   
974                       dhalander.com.br                0          0   
975                        classified38.ir                0          0   
976                      jnep.sumdu.edu.ua                0          0   
977                  confifsk.sumdu.edu.ua                0          0   
978                          www.gem.tools                0          0   
979                           sunnsand.biz                0          0   
980                 garrisoncontracting.ca                0          0   
981                              hecmk.com                0          0   
982                          unione-ae.com                0          0   
983                            payasgo.org                1          0   
984         kalempong-godang-2017.16mb.com                0          0   
985                unblocking-notice.co.nf                0          0   
986         fb-secure-notifications.hol.es                0          0   
987          radiocatolicalaconsentida.com                1          0   
988              mobile-free.kofoed-fr.com                0          0   
989                       mashtaresort.com                0          0   
990              mobile-free.kofoed-fr.c

In [11]:
#data.to_csv("legitimate-urls.csv",index=False,encoding='UTF-8')

In [90]:
data.to_csv("phishing-urls.csv",index=False,encoding='UTF-8')